In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set the style for nicer plots
sns.set_theme(style="whitegrid")

In [ ]:
filename = 'comparative_study_results.csv'
data_dir = 'simulation_data'
# Replace 'your_results.csv' with your actual filename
df = pd.read_csv(os.path.join(data_dir, filename))

# Display basic info to verify
print(f"Loaded {len(df)} rows.")
display(df.head())

In [ ]:
# Get list of unique graph names
unique_graphs = df['graph_name'].unique()


for graph in unique_graphs:
    unique_r_values = df[df['graph_name'] == graph]['r'].unique()
    for r in unique_r_values:
        # 1. Filter data for this specific graph
        subset = df[(df['graph_name'] == graph) & (df['r'] == r)]
        N = subset['N'].iloc[0]
        # 2. Filter for only successful fixations
        fixation_subset = subset[subset['fixation'] == True]
        
        # 3. Calculate Probability of Fixation
        p_fix = subset['fixation'].mean()
        n_runs = len(subset)
        n_success = len(fixation_subset)
        
        # --- PLOTTING ---
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Main Title with Statistics
        fig.suptitle(f"Graph: {graph}|N: {N} |r: {r} | Runs: {n_runs} | P(fix): {p_fix:.3f} ({n_success}/{n_runs})", 
                    fontsize=14, fontweight='bold', y=1.05)
        
        # Plot 1: Histogram of Steps (All runs: Fixation + Extinction)
        sns.histplot(data=subset, x='steps', hue='fixation', multiple="stack", 
                    ax=axes[0], palette={True: 'green', False: 'gray'})
        median_steps = subset['steps'].median()
        axes[0].set_title(f"Distribution of Steps (All Runs) — median: {median_steps:.0f} steps")
        axes[0].set_xlabel("Steps")
        axes[0].set_ylabel("Frequency")
        
        # Plot 2: Histogram of Steps (Only Fixation=True)
        if not fixation_subset.empty:
            sns.histplot(data=fixation_subset, x='steps', ax=axes[1], color='green', kde=True)
            median_steps_fixed = fixation_subset['steps'].median()
            axes[1].set_title(f"Distribution of Steps (Only Fixation Events) — median: {median_steps_fixed:.0f} steps")
            axes[1].set_xlabel("Steps to Fixation")
            axes[1].set_ylabel("Frequency")
        else:
            # Handle case where no fixations occurred
            axes[1].text(0.5, 0.5, "No Fixation Events Observed", 
                        horizontalalignment='center', verticalalignment='center', fontsize=12)
            axes[1].set_title("Distribution of Steps (Only Fixation Events)")

        plt.tight_layout()
        plt.show()

In [ ]:
# 1. Filter Data: Remove 'Cycle' graphs and prepare the dataset
# We use str.contains to catch "Cycle", "cycle_graph", etc.
df_clean = df[~df['graph_name'].str.contains("cycle", case=False, na=False)]

# ---------------------------------------------------------
# TABLE 1: Median Steps (Only Successful Fixations)
# ---------------------------------------------------------
fixation_df = df_clean[df_clean['fixation'] == True]

table_fixation = fixation_df.pivot_table(
    index=['graph_name', 'N'],  # Group by Name AND N
    columns='r', 
    values='steps', 
    aggfunc='median'
)

# Sort by Row Average:
# 1. Calculate mean across columns (axis=1)
# 2. Sort the DataFrame by that mean
# 3. Drop the temporary mean column so it doesn't show in the final table
table_fixation = table_fixation.assign(row_mean=table_fixation.mean(axis=1)) \
                               .sort_values('row_mean') \
                               .drop(columns='row_mean')

print("--- Table 1: Median Steps (Only Successful Fixations) [Sorted by Row Avg] ---")
display(table_fixation)

# ---------------------------------------------------------
# TABLE 2: Median Steps (All Runs)
# ---------------------------------------------------------
table_all = df_clean.pivot_table(
    index=['graph_name', 'N'], 
    columns='r', 
    values='steps', 
    aggfunc='median'
)

# Sort by Row Average
table_all = table_all.assign(row_mean=table_all.mean(axis=1)) \
                     .sort_values('row_mean') \
                     .drop(columns='row_mean')

print("\n--- Table 2: Median Steps (All Runs) [Sorted by Row Avg] ---")
display(table_all)

In [ ]:
# 1. Filter Data: Remove 'Cycle' graphs
df_clean = df[~df['graph_name'].str.contains("cycle", case=False, na=False)]

# ---------------------------------------------------------
# TABLE 3: Fixation Probability (P_fix)
# ---------------------------------------------------------
table_prob = df_clean.pivot_table(
    index=['graph_name', 'N'], 
    columns='r', 
    values='fixation', 
    aggfunc='mean'  # Mean of boolean = Probability
)

# Sort by Row Average (Descending: Highest probability at the top)
table_prob = table_prob.assign(row_mean=table_prob.mean(axis=1)) \
                       .sort_values('row_mean', ascending=False) \
                       .drop(columns='row_mean')

print("--- Table 3: Fixation Probability (P_fix) [Sorted by High -> Low] ---")
display(table_prob)

In [ ]:
df_clean = df[~df['graph_name'].str.contains("cycle", case=False, na=False)]

# 2. Aggregation Logic
# We need to compute stats for each (Graph, r) pair independently

# Metric A: Fixation Probability (Using all data)
# Group by graph and r, then take mean of 'fixation' (True/False -> 1/0)
prob_df = df_clean.groupby(['graph_name', 'r'])['fixation'].mean().reset_index(name='prob_fixation')

# Metric B: Median Steps (Using only SUCCESSFUL fixations)
# We filter first, because we don't want extinction times contaminating this metric
success_only_df = df_clean[df_clean['fixation'] == True]
time_df = success_only_df.groupby(['graph_name', 'r'])['steps'].median().reset_index(name='median_steps')

# Merge metrics into one plotting table
plot_data = pd.merge(prob_df, time_df, on=['graph_name', 'r'])

# 3. Create Scatter Plot
plt.figure(figsize=(10, 7))
sns.scatterplot(
    data=plot_data,
    x='median_steps',
    y='prob_fixation',
    hue='graph_name',  # Different colors for different graphs
    style='r',         # Different shapes for different r values
    s=150,             # Make points larger
    palette='deep',    # Nice color palette
    alpha=0.9
)

# 4. Styling
plt.title('Trade-off: Fixation Probability vs Time', fontsize=15)
plt.xlabel('Median Steps to Fixation (Condition: Success)', fontsize=12)
plt.ylabel('Probability of Fixation', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)

# Move legend outside to keep plot clean
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

plt.tight_layout()
plt.show()